# Input

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import xpress as xp
xp.controls.outputlog = 0

In [2]:
T = 168

In [3]:
N = 100

In [4]:
bornes_random = [np.sort(np.random.randint(low=0,high=10,size=(T,2)),axis=1) for i in range(N)]
# for i in range(N):
#     bornes_random[i][:,0] = np.zeros(T)

In [5]:
production_random = [np.random.rand(T)*10 for i in range(N)]

In [6]:
delta = [randint(1,40) for i in range(N)]

In [7]:
len(production_random[0])

168

In [8]:
def main_problem(on_min, on_max, d, prod):
    p = xp.problem()
    p.controls.xslp_log = -1

    n = [xp.var(lb=on_min[i],ub=on_max[i]) for i in range(T)]
    n_plus = [xp.var(lb=0) for i in range(T)]
    n_moins = [xp.var(lb=0) for i in range(T)]
    n_moins_moins = [xp.var(lb=0,ub=max(0,on_max[i-1]-on_max[i])) for i in range(T)]
    unsupplied = [xp.var(lb=0) for i in range(T)]

    p.addVariable(n,n_plus,n_moins,n_moins_moins,unsupplied)

    p.addConstraint(n[i]-n[i-1] == n_plus[i]-n_moins[i] for i in range(1,T))
    p.addConstraint(n_moins[i]>=n_moins_moins[i] for i in range(T))
    p.addConstraint(n[i]>=xp.Sum([n_plus[k]-n_moins_moins[k] for k in range(i-d+1,i+1)]) for i in range(T))
    p.addConstraint(n[i]<=on_max[i - d] - xp.Sum([n_moins[k]-max(0,on_max[k]-on_max[k-1]) for k in range(i-d+1,i+1)]) for i in range(T))
    p.addConstraint(unsupplied[i]>=prod[i]-n[i] for i in range(T))

    p.setObjective(xp.Sum(unsupplied))

    p.solve()

    assert p.getProbStatus()==1

    return p.getSolution(n,n_plus,n_moins,n_moins_moins)

In [9]:
def heuristic_problem(on_min, on_max, d,debug=False):
    p = xp.problem()
    p.controls.xslp_log = -1

    n = [xp.var(name=f"n_{i}",lb=on_min[i],ub=on_max[i]) for i in range(T)]
    n_plus = [xp.var(name=f"n_plus_{i}",lb=0) for i in range(T)]
    n_moins = [xp.var(name=f"n_moins_{i}",lb=0) for i in range(T)]
    n_moins_moins = [xp.var(name=f"n__moins_moins{i}",lb=0,ub=max(0,on_max[i-1]-on_max[i])) for i in range(T)]

    p.addVariable(n,n_plus,n_moins,n_moins_moins)

    p.addConstraint(n[i]-n[i-1] == n_plus[i]-n_moins[i] for i in range(1,T))
    p.addConstraint(n_moins[i]>=n_moins_moins[i] for i in range(T))
    p.addConstraint(n[i]>=xp.Sum([n_plus[k]-n_moins_moins[k] for k in range(i-d+1,i+1)]) for i in range(T))
    p.addConstraint(n[i]<=on_max[i - d] - xp.Sum([n_moins[k]-max(0,on_max[k]-on_max[k-1]) for k in range(i-d+1,i+1)]) for i in range(T))

    p.setObjective(xp.Sum([n[i] for i in range (T)]))

    if debug:
        p.write("p", "lp")

    p.solve()

    assert p.getProbStatus()==1

    return p.getSolution(n,n_plus,n_moins,n_moins_moins)

# Itération 1

In [10]:
itr_1 = []
for i in range(N):
    itr_1.append(main_problem(bornes_random[i][:,0],bornes_random[i][:,1],delta[i], production_random[i]))

In [11]:
for i in range(N):
    for j in range(4):
        assert np.all(np.abs(np.round(itr_1[i][j])-itr_1[i][j])<=1e-5)

AssertionError: 

In [12]:
print(i, j )

0 0


In [13]:
np.where(np.abs(np.round(itr_1[i][j])-itr_1[i][j])>1e-5)

(array([  2,  11,  12,  18,  19,  22,  23,  28,  32,  33,  34,  36,  37,
         59,  60,  62,  77,  91,  92,  98,  99, 102, 103, 110, 111, 116,
        117, 118, 127, 128, 129, 130, 131, 133, 139, 145, 146, 147, 166,
        167], dtype=int64),)

In [14]:
itr_1[i][j][:10]

[6.0, 6.0, 7.477442564799435, 7.0, 0.0, 6.0, 5.0, 7.0, 7.0, 8.0]

# Ceil itération 1

In [15]:
contraintes_respectées = [heuristic_problem(np.ceil(np.round(itr_1[i][0],12)),np.ceil(np.round(itr_1[i][0],12)),delta[i]) for i in range(N)]

In [16]:
for i in range(N):
    for j in range(4):
        assert np.all(np.abs(np.round(contraintes_respectées[i][j],decimals=0)-contraintes_respectées[i][j])<=1e-5)

# Heuristique

In [17]:
heuristic = [heuristic_problem(np.ceil(np.round(itr_1[i][0],12)),bornes_random[i][:,1],delta[i]) for i in range(N)]

In [18]:
for i in range(N):
    for j in range(4):
        assert np.all(np.abs(np.round(heuristic[i][j])-heuristic[i][j])<=1e-5)

# Heuristique sur aléatoire

In [21]:
heuristic_on_random = [heuristic_problem(bornes_random[i][:,0],bornes_random[i][:,1],delta[i]) for i in range(N)]

In [22]:
for i in range(N):
    for j in range(4):
        assert np.all(np.abs(np.round(heuristic_on_random[i][j])-heuristic_on_random[i][j])<=1e-5)

AssertionError: 

In [23]:
print(i, j)

13 0


In [24]:
np.where(np.abs(np.round(heuristic_on_random[i][j])-heuristic_on_random[i][j])>1e-5)

(array([105, 120, 121, 123, 126], dtype=int64),)

In [26]:
heuristic_on_random[i][j][120:130]

[2.5, 2.5, 8.0, 2.5, 5.0, 7.0, 6.5, 6.0, 4.0, 3.0]